# Modèle 1 : Implementation sentence transformers étude complementaire (Architecture 3)

Dans l'étude précédente nous nous avons fais ces choix :

- **Sens sémentique de la requête de l'utilisateur :** Nous avons fais pareil que pour les autres études, nous calculons un vecteur pour chaque phrases et nous faisons la moyenne. L'idée était d'avoir le sens sémentique globale de la requête.

- **Sens sémentique des solutions :** Comparer chaque section indépendament avec l'entrée de l'utilisateur et ne garder que le max de la correspondance sémentique. (Le titre, la description, etc)

## Imports

In [ ]:
!pip install pandas spacy bs4 sentence_transformers numpy
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 3.5 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [3]:
import pandas as pd
import re
import unicodedata
import spacy
from bs4 import BeautifulSoup
from spacy.lang.fr.stop_words import STOP_WORDS
import pickle
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.quantization import quantize_embeddings
import numpy as np

/home/patrice/Documents/Etudes/CY-Tech/Ing2_Sem2_2023-2024/UE7_Projet_DataBattle_IA-Pau/ia-pau-data-battle-mars-2024/backend/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Fonctions inchangées de l'ancienne étude

In [4]:
def calculate_average_embedding(text, quantize=False, precision="binary"):
    # Diviser le texte en phrases
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
    
    # Calculer l'embedding de chaque phrase
    if quantize :
        sentence_embeddings = model.encode(sentences, precision=precision)
    else :
        sentence_embeddings = model.encode(sentences)
    
    # Prendre la moyenne des embeddings des phrases
    if len(sentence_embeddings) > 0:
        average_embedding = np.mean(sentence_embeddings, axis=0)
    else:
        average_embedding = np.zeros(model.get_sentence_embedding_dimension())
    
    return average_embedding

def pre_processing(texte):
    # Nettoyer HTML Tags
    texte = BeautifulSoup(texte, 'html.parser').get_text()

    # Remplacer "&nbsp;." par rien
    texte = re.sub(r'&nbsp;\.', '', texte)

    # Accents
    texte = unicodedata.normalize('NFD', texte).encode('ascii', 'ignore').decode("utf-8")

    # Retirer les numéros
    texte = re.sub(r'\b\d+\b', '', texte)

    # Tokenization, Lemmatization, Removing Stopwords, Lowercase
    doc = nlp(texte)
    phrases_propres = []
    for phrase in doc.sents:
        tokens = [token.lemma_.lower() for token in phrase if not token.is_stop and not token.is_punct and not token.is_space]
        phrase_propre = ' '.join(tokens)
        if phrase_propre:
            phrases_propres.append(phrase_propre + ".")  # Ajouter un point à la fin de la phrase propre

    # Joining the cleaned sentences back into a single string
    cleaned_text = ' '.join(phrases_propres)

    return cleaned_text

## Fonctions adaptées

In [5]:
def pre_processing_sentences(sentences):
    phrases_propres = []

    for texte in sentences:
        # Nettoyer HTML Tags
        texte = BeautifulSoup(texte, 'html.parser').get_text()

        # Remplacer "&nbsp;." par rien
        texte = re.sub(r'&nbsp;\.', '', texte)

        # Accents
        texte = unicodedata.normalize('NFD', texte).encode('ascii', 'ignore').decode("utf-8")

        # Retirer les numéros
        texte = re.sub(r'\b\d+\b', '', texte)

        # Tokenization, Lemmatization, Removing Stopwords, Lowercase
        doc = nlp(texte)
        for phrase in doc.sents:
            tokens = [token.lemma_.lower() for token in phrase if not token.is_stop and not token.is_punct and not token.is_space]
            phrase_propre = ' '.join(tokens)
            if phrase_propre:
                phrases_propres.append(phrase_propre)
    return phrases_propres

In [6]:
def load_and_merge_data(csv_file = '../data/solutions.csv'):
    # Charger le fichier CSV en spécifiant le séparateur '|'
    df = pd.read_csv(csv_file, sep='|', header=None)
    # Renommer les colonnes
    df.columns = ['id_solution', 'categorie', 'texte']
    # Filtrer les lignes pour les catégories spécifiées
    categories_specifiees = [1, 2, 5, 6]
    df_filtre = df[df['categorie'].isin(categories_specifiees)]
    # Pivoter les données pour obtenir les colonnes 'titre', 'definition', 'application' et 'bilan énergie'
    df_pivot = df_filtre.pivot(index='id_solution', columns='categorie', values='texte').reset_index()
    # Renommer les colonnes
    df_pivot.columns = ['id_solution', 'titre', 'definition', 'application', 'bilan_energie']

    # Gérer les valeurs NaN lors de la fusion des colonnes
    def combine_text(row):
        text_parts = [row[col] for col in colonnes if pd.notnull(row[col])]
        return text_parts
    
    # Sélectionner uniquement les colonnes 'id_solution' et les champs requis
    colonnes = ['titre', 'definition', 'application', 'bilan_energie']
    df_pivot['champs'] = df_pivot.apply(combine_text, axis=1)
    df_final = df_pivot[['id_solution', 'champs']]
    
    # Convertir en liste de listes pour chaque ligne
    result = df_final.values.tolist()
    
    return result

In [7]:
# NOUVELLE FONCTION
def calculate_section_embedding(sentences, quantize=False, precision="binary"):
    
    # Calculer l'embedding de chaque phrase
    if quantize :
        sentence_embeddings = model.encode(sentences, precision=precision)
    else :
        sentence_embeddings = model.encode(sentences)
    
    return sentence_embeddings

In [8]:
def genere_embedding(data, output_file, quantize=False, precision="binary"):

    # Appliquer la fonction pour calculer l'embedding moyen à chaque texte
    if quantize:
        embeddings = data['clean_text'].apply(calculate_section_embedding, quantize=True)
    else :
        embeddings = data['clean_text'].apply(calculate_section_embedding)

    
    # Créer un nouveau DataFrame avec id_solution et les embeddings
    new_data = {
        'id_solution': data['id_solution'],
        'text_embedding': embeddings # Désormais mes embeddings sont des listes de solutions contenant chacune les vecteurs de toutes les phrases de la solution.
    }
    
    # Créer un DataFrame à partir des nouvelles données
    df_embeddings = pd.DataFrame(new_data)
    
    # Storer les embeddings dans un fichier
    with open(output_file, "wb") as fOut:
        pickle.dump(df_embeddings, fOut, protocol=pickle.HIGHEST_PROTOCOL)
    
    print("Les embeddings ont été storer avec succès.")

In [9]:
def find_solution(text_to_compare, embeddings_file, quantize=False, precision="binary"):
    # Calculer l'embedding moyen du texte à comparer
    embedding_to_compare = calculate_average_embedding(text_to_compare, quantize, precision)
    
    # Charger les embeddings à partir du fichier
    with open(embeddings_file, "rb") as fIn:
        df_embeddings = pickle.load(fIn)


    list_similarities = []
    # Pour chaque solution
    for solution in df_embeddings['text_embedding'].values:
        embeddings_array = np.stack(solution)
        # Calculer la similarité cosinus entre l'embedding à comparer et les embeddings dans df_embeddings
        similarities = util.pytorch_cos_sim(embedding_to_compare.reshape(1, -1), embeddings_array)

        max_value, _ = similarities.max(dim=1)
        list_similarities.append(max_value)

    # Ajouter les similarités au DataFrame df_embeddings
    df_embeddings['similarity'] = list_similarities
    
    # Trier par similarité décroissante
    df_sorted = df_embeddings.sort_values(by='similarity', ascending=False)
    
    # Récupérer les id_solution et les similarités
    solution_info = df_sorted[['id_solution', 'similarity']].head(10)
    
    # Convertir en liste de tuples (id_solution, similarity)
    solution_list = list(zip(solution_info['id_solution'], solution_info['similarity']))
    
    return solution_list

## Tests

In [20]:
df_solutions = load_and_merge_data()

# Convertir la liste de listes en DataFrame pandas
df_solutions = pd.DataFrame(df_solutions, columns=['id_solution', 'text'])

# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")

# Appliquer le traitement à la colonne "text" de notre df_solutions
df_solutions['clean_text'] = df_solutions['text'].apply(pre_processing_sentences)

# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

/tmp/ipykernel_2832/3606916284.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  texte = BeautifulSoup(texte, 'html.parser').get_text()


Génère embedding

In [48]:
# Générons notre embedding
genere_embedding(df_solutions, "Architecure-3_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl")

Les embeddings ont été storer avec succès.


Test inférence

In [11]:
# Malheureusement nous n'avons pas reçu le domaine d'activité correspondant aux 
# requetes avec Kerdos.Et pour certaines requetes nous n'avons pas l'Id_solution non
# plus, dans ce cas il est remplacé par -1.
dataset_test_kerdos = [
    ["Id_solution", "Domaine_activite", "Description"],
    [54, "", "Moteur à haut rendement"], # Titre de sol 54
    [156, "", "Remplacement des outils pneumatiques par des outils électriques lorsque c'est possible "], # Titre de sol 156
    [724, "","C'est quoi la HP flottante ?"],
    [914, "", "Je voudrais dimensionner un panneau solaire."],
    [719, "", "Quel gain pour un variateur de vitesse ?"],
    [-1, "", "Comment faire pour réduire la consommation de mon compresseur d'air comprimé ?"],
    [-1, "", "J'aimerais avoir une régulation optimisée de mon groupe froid"],
    [-1, "", "Quelles sont les meilleures solutions pour l'agro-alimentaire ?"],
    [-1, "", "Comment faire pour réduire la consommation de mon compresseur d'air comprimé ?"]
]

In [12]:
# Fonction appelé par notre utilisateur
def model_PAT(secteur, description) :
    # On commence par concaténer notre secteur et notre description.
    text = secteur + ". " + description

    # Ensuite on applique notre pré-processing
    clean_text = pre_processing(text)

    # Ensuite on cherche nos similarités 
    solutions = find_solution(clean_text, "embeddings/Architecure-3_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl")

    # On return une liste contenant uniquement le numéros des solutions
    id_solutions = []
    for solution in solutions :
        id_solutions.append(solution[0])

    return id_solutions

In [13]:
# On va tester sur notre dataset_test
for i in range(1,len(dataset_test_kerdos)):
    print("--------------------------------------------")
    print("Solution attendue : ", dataset_test_kerdos[i][0])
    print(model_PAT(dataset_test_kerdos[i][1], dataset_test_kerdos[i][2]))

--------------------------------------------
Solution attendue :  54
[54, 1070, 328, 789, 55, 61, 58, 480, 1445, 51]
--------------------------------------------
Solution attendue :  156
[156, 157, 1056, 415, 1050, 919, 58, 32, 77, 853]
--------------------------------------------
Solution attendue :  724
[724, 1129, 868, 1061, 1533, 388, 823, 356, 1595, 299]
--------------------------------------------
Solution attendue :  914
[1454, 1457, 1452, 1453, 1455, 1456, 368, 1486, 914, 818]
--------------------------------------------
Solution attendue :  719
[63, 507, 1142, 1445, 356, 1647, 1649, 1144, 1639, 1609]
--------------------------------------------
Solution attendue :  -1
[162, 1056, 145, 1001, 332, 1051, 1058, 357, 165, 159]
--------------------------------------------
Solution attendue :  -1
[1049, 734, 12, 1693, 191, 483, 103, 148, 790, 803]
--------------------------------------------
Solution attendue :  -1
[1630, 1099, 1522, 749, 3, 804, 1560, 745, 1025, 1007]
--------------

# Test patrice_test_set

In [30]:
# Lire le fichier CSV
df_testset = pd.read_csv("data/patrice_test_set.csv")

def test_accuracy(dataset=df_testset, top_n=1) :
    accuracy = 0
    for i in range(1,len(dataset)):
        predictions = model_PAT("", dataset['Description'][i])
        if (dataset["id_solution"][i] in predictions[:top_n]):
            accuracy += 1/len(dataset)
    return accuracy

In [31]:
test_accuracy()

0.7500000000000003